In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from models import *

from os import listdir
from os.path import isfile, join
import numpy as np
import math

import torch.optim as optim

# Define from where to load the files 

In [2]:
polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

# Load the data

In [3]:
grid_size = 32
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=False)

# Build a training and validation set 

In [4]:
batch_size = 10 
validation_size=50
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target, validation_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:,0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])
image_size = 32

train_input = np.array(preprocessed_input_train.view(Ntrain, 1,image_size, image_size, image_size))
validation_input = np.array(preprocessed_input_validation.view(Nvalidation, 1,image_size, image_size, image_size))

labels_train = np.array(preprocessed_input_train_target.view(Ntrain, 1, image_size, image_size, image_size))
labels_validation = np.array(preprocessed_input_validation_target.view(Nvalidation, 1,image_size, image_size, image_size))

# Create the target 
Find the middle of the cube and the distance to the x, y and z boundaries

In [5]:
labels_train_cube_coords = np.zeros((len(train_input[:,0,0,0,0]), 1, 9))
for i in range (len(train_input[:,0,0,0,0])):
    #solution 1 - the loss is calculed based on the 9 labels 
    delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z = find_center_and_delta(labels_train[i, 0, :, :, :], grid_size=32)
    labels_train_cube_coords[i,0,0] = delta_x_left
    labels_train_cube_coords[i,0,1] = delta_x_right
    labels_train_cube_coords[i,0,2] = center_x
    labels_train_cube_coords[i,0,3] = delta_y_left
    labels_train_cube_coords[i,0,4] = delta_y_right
    labels_train_cube_coords[i,0,5] = center_y
    labels_train_cube_coords[i,0,6] = delta_z_left
    labels_train_cube_coords[i,0,7] = delta_z_right
    labels_train_cube_coords[i,0,8] = center_z
    
labels_validation_cube_coords = np.zeros((len(labels_validation[:,0,0,0,0]), 1, 9))
for i in range (len((validation_input[:,0,0,0,0]))):
    #solution 1 - the loss is calculed based on the 9 labels 
    delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z = find_center_and_delta(labels_train[i, 0, :, :, :], grid_size=32)
    labels_validation_cube_coords[i, 0,0] = delta_x_left
    labels_validation_cube_coords[i, 0,1] = delta_x_right
    labels_validation_cube_coords[i, 0,2] = center_x
    labels_validation_cube_coords[i, 0,3] = delta_y_left
    labels_validation_cube_coords[i, 0,4] = delta_y_right
    labels_validation_cube_coords[i, 0,5] = center_y
    labels_validation_cube_coords[i, 0,6] = delta_z_left
    labels_validation_cube_coords[i, 0,7] = delta_z_right
    labels_validation_cube_coords[i, 0,8] = center_z
    
    
labels_validation_cube_coords = torch.from_numpy(labels_validation_cube_coords)
labels_train_cube_coords = torch.from_numpy(labels_train_cube_coords)
train_input = torch.from_numpy(train_input)
validation_input = torch.from_numpy(validation_input)

print('train_input', train_input.shape)
print('labels_train_cube_coords', labels_train_cube_coords.shape)

print('validation_input', validation_input.shape)
print('labels_validation_cube_coords', labels_validation_cube_coords.shape)

    
    #solution 2 - the loss is calculated from the cube generated thanks to the learned coordinates 
    #output = build_cube(delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z, grid_size)
    

train_input torch.Size([550, 1, 32, 32, 32])
labels_train_cube_coords torch.Size([550, 1, 9])
validation_input torch.Size([50, 1, 32, 32, 32])
labels_validation_cube_coords torch.Size([50, 1, 9])


In [6]:
# Train network 
#criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss()
criterion = nn.MSELoss()

train_input = Variable(train_input).float()
validation_input = Variable(validation_input).float()

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(labels_train_cube_coords)  # keep long tensors
    validation_target = Variable(labels_validation_cube_coords, 
    uires_grad=False) # convert to float
    Noutputs = 18
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(labels_train_cube_coords)  # keep long tensors
    validation_target = Variable(labels_validation_cube_coords, requires_grad=False) # convert to float
    Noutputs = 18
    
else:
    train_target = Variable(labels_train_cube_coords, requires_grad=False).float() # convert to float
    validation_target = Variable(labels_validation_cube_coords, requires_grad=False).float() # convert to float
    Noutputs = 9

Nbatches = int(math.ceil(Ntrain/batch_size))
Nepochs = 1000
#seeds = list(range(15)) #Test 15 different seeds but always the seeds from 0 to 15 so that the weights are always initialized in a reproducible way
#Nrep = len(seeds)
Nrep = 1

train_errors = torch.Tensor(Nrep, Nepochs).zero_()
test_errors = torch.Tensor(Nrep, Nepochs).zero_()
validation_errors = torch.Tensor(Nrep, Nepochs).zero_()
ep_loss = torch.Tensor(Nrep, Nepochs).zero_()

for i_rep in range(Nrep):    
    #print('Repetition', seeds[i_rep])
    #torch.manual_seed(seeds[i_rep])
    
    model = conv2DNet_1(Noutputs) #from litterature EEG-Net coorected
    #model = conv2DNet_2(Noutputs)  #from Temporal - Spatial; 4 Filters Model - Best performing model with accuracy 0.83 in average on the validation set
    #model = conv2DNet_3(Noutputs) #from Temporal - Spatial; 64 Filters Model
    #model = conv2DNet_4(Noutputs) #from Temporal - Spatial; 128 Filters Model
    
    #optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.50)
    optimizer = optim.Adam(model.parameters())
    #optimizer = optim.Adagrad(model.parameters())
    #optimizer = optim.Adamax(model.parameters())
    #optimizer = optim.ASGD(model.parameters())
    #optimizer = optim.RMSprop(model.parameters())
    #optimizer = optim.Rprop(model.parameters())
    
    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True) #Reduces the learning rate if it did not decreased by more than 10^-4 in 10 steps

    for i_ep in range(Nepochs):
        for b_start in range(0, Ntrain, batch_size):
            bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
            model.train()
            model.zero_grad()
            output = model(train_input.narrow(0, b_start, bsize_eff))
            if isinstance(criterion, nn.CrossEntropyLoss) or isinstance(criterion, nn.NLLLoss):
                batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))
            else:
                batch_loss = criterion(output.view(bsize_eff*Noutputs), train_target.narrow(0, b_start, bsize_eff))
            ep_loss[i_rep, i_ep] += batch_loss.data[0]
            batch_loss.backward()
            optimizer.step()
        
        #scheduler.step(ep_loss[i_rep, i_ep])
        
        nb_train_errs = compute_nb_errors_delta(model, train_input, train_target, batch_size, criterion)
        nb_validation_errs = compute_nb_errors_delta(model, validation_input, validation_target, batch_size, criterion)
        
        print("Epoch Number : ", i_ep)
        print("\t Training accuracy: ", (100*(Ntrain*Noutputs-nb_train_errs)/(Ntrain*Noutputs)))
        print("\t Validation accuracy ",(100*(Nvalidation*Noutputs-nb_validation_errs)/(Nvalidation*Noutputs)))
        
        print("\t Epoch Loss ", ep_loss[i_rep, i_ep])
        
        train_errors[i_rep, i_ep] = nb_train_errs
        validation_errors[i_rep, i_ep] = nb_validation_errs

Epoch Number :  0
	 Training accuracy:  98.58585858585859
	 Validation accuracy  85.11111111111111
	 Epoch Loss  2527.916748046875
Epoch Number :  1
	 Training accuracy:  98.78787878787878
	 Validation accuracy  86.44444444444444
	 Epoch Loss  227.7554168701172
Epoch Number :  2
	 Training accuracy:  98.78787878787878
	 Validation accuracy  86.44444444444444
	 Epoch Loss  159.5004119873047
Epoch Number :  3
	 Training accuracy:  99.01010101010101
	 Validation accuracy  85.55555555555556
	 Epoch Loss  147.3355712890625
Epoch Number :  4
	 Training accuracy:  98.96969696969697
	 Validation accuracy  84.66666666666667
	 Epoch Loss  162.86672973632812
Epoch Number :  5
	 Training accuracy:  98.76767676767676
	 Validation accuracy  85.11111111111111
	 Epoch Loss  178.31932067871094
Epoch Number :  6
	 Training accuracy:  98.56565656565657
	 Validation accuracy  85.55555555555556
	 Epoch Loss  186.02066040039062
Epoch Number :  7
	 Training accuracy:  98.56565656565657
	 Validation accuracy 

Epoch Number :  63
	 Training accuracy:  99.55555555555556
	 Validation accuracy  84.66666666666667
	 Epoch Loss  30.179521560668945
Epoch Number :  64
	 Training accuracy:  99.41414141414141
	 Validation accuracy  85.33333333333333
	 Epoch Loss  32.2234001159668
Epoch Number :  65
	 Training accuracy:  99.27272727272727
	 Validation accuracy  85.77777777777777
	 Epoch Loss  30.080976486206055
Epoch Number :  66
	 Training accuracy:  99.47474747474747
	 Validation accuracy  86.66666666666667
	 Epoch Loss  31.45897674560547
Epoch Number :  67
	 Training accuracy:  99.0909090909091
	 Validation accuracy  85.55555555555556
	 Epoch Loss  33.778629302978516
Epoch Number :  68
	 Training accuracy:  99.23232323232324
	 Validation accuracy  85.11111111111111
	 Epoch Loss  34.22681427001953
Epoch Number :  69
	 Training accuracy:  99.25252525252525
	 Validation accuracy  85.55555555555556
	 Epoch Loss  35.510528564453125
Epoch Number :  70
	 Training accuracy:  99.47474747474747
	 Validation ac

Epoch Number :  125
	 Training accuracy:  99.31313131313131
	 Validation accuracy  85.11111111111111
	 Epoch Loss  21.87447738647461
Epoch Number :  126
	 Training accuracy:  99.31313131313131
	 Validation accuracy  85.77777777777777
	 Epoch Loss  22.189990997314453
Epoch Number :  127
	 Training accuracy:  99.5959595959596
	 Validation accuracy  85.11111111111111
	 Epoch Loss  20.57637596130371
Epoch Number :  128
	 Training accuracy:  99.35353535353535
	 Validation accuracy  85.77777777777777
	 Epoch Loss  21.26285171508789
Epoch Number :  129
	 Training accuracy:  99.53535353535354
	 Validation accuracy  84.88888888888889
	 Epoch Loss  21.349018096923828
Epoch Number :  130
	 Training accuracy:  99.17171717171718
	 Validation accuracy  86.88888888888889
	 Epoch Loss  15.959776878356934
Epoch Number :  131
	 Training accuracy:  99.43434343434343
	 Validation accuracy  86.88888888888889
	 Epoch Loss  17.102689743041992
Epoch Number :  132
	 Training accuracy:  99.43434343434343
	 Vali

Epoch Number :  188
	 Training accuracy:  99.43434343434343
	 Validation accuracy  86.22222222222223
	 Epoch Loss  16.130935668945312
Epoch Number :  189
	 Training accuracy:  99.61616161616162
	 Validation accuracy  86.88888888888889
	 Epoch Loss  19.67966079711914
Epoch Number :  190
	 Training accuracy:  99.51515151515152
	 Validation accuracy  86.66666666666667
	 Epoch Loss  18.032135009765625
Epoch Number :  191
	 Training accuracy:  99.1919191919192
	 Validation accuracy  86.22222222222223
	 Epoch Loss  18.487520217895508
Epoch Number :  192
	 Training accuracy:  99.31313131313131
	 Validation accuracy  85.55555555555556
	 Epoch Loss  23.160755157470703
Epoch Number :  193
	 Training accuracy:  99.45454545454545
	 Validation accuracy  85.11111111111111
	 Epoch Loss  26.660385131835938
Epoch Number :  194
	 Training accuracy:  99.77777777777777
	 Validation accuracy  85.33333333333333
	 Epoch Loss  14.964457511901855
Epoch Number :  195
	 Training accuracy:  99.51515151515152
	 Va

Epoch Number :  250
	 Training accuracy:  98.94949494949495
	 Validation accuracy  85.77777777777777
	 Epoch Loss  27.197240829467773
Epoch Number :  251
	 Training accuracy:  98.62626262626263
	 Validation accuracy  84.0
	 Epoch Loss  35.50455856323242
Epoch Number :  252
	 Training accuracy:  99.23232323232324
	 Validation accuracy  86.66666666666667
	 Epoch Loss  38.90116882324219
Epoch Number :  253
	 Training accuracy:  99.37373737373737
	 Validation accuracy  85.33333333333333
	 Epoch Loss  25.944936752319336
Epoch Number :  254
	 Training accuracy:  99.1919191919192
	 Validation accuracy  84.88888888888889
	 Epoch Loss  28.16469955444336
Epoch Number :  255
	 Training accuracy:  99.37373737373737
	 Validation accuracy  88.44444444444444
	 Epoch Loss  25.675315856933594
Epoch Number :  256
	 Training accuracy:  99.41414141414141
	 Validation accuracy  86.44444444444444
	 Epoch Loss  24.643434524536133
Epoch Number :  257
	 Training accuracy:  99.4949494949495
	 Validation accurac

...

Epoch Number :  687
	 Training accuracy:  99.51515151515152
	 Validation accuracy  86.88888888888889
	 Epoch Loss  10.88304615020752
Epoch Number :  688
	 Training accuracy:  99.81818181818181
	 Validation accuracy  87.11111111111111
	 Epoch Loss  7.901096820831299
Epoch Number :  689
	 Training accuracy:  99.67676767676768
	 Validation accuracy  87.11111111111111
	 Epoch Loss  6.550812244415283
Epoch Number :  690
	 Training accuracy:  99.85858585858585
	 Validation accuracy  87.11111111111111
	 Epoch Loss  5.627173900604248
Epoch Number :  691
	 Training accuracy:  99.79797979797979
	 Validation accuracy  86.66666666666667
	 Epoch Loss  5.406783580780029
Epoch Number :  692
	 Training accuracy:  99.87878787878788
	 Validation accuracy  86.66666666666667
	 Epoch Loss  5.344302654266357
Epoch Number :  693
	 Training accuracy:  99.87878787878788
	 Validation accuracy  87.33333333333333
	 Epoch Loss  5.097585678100586
Epoch Number :  694
	 Training accuracy:  99.87878787878788
	 Validat

Epoch Number :  750
	 Training accuracy:  99.47474747474747
	 Validation accuracy  84.88888888888889
	 Epoch Loss  17.24879264831543
Epoch Number :  751
	 Training accuracy:  98.96969696969697
	 Validation accuracy  84.44444444444444
	 Epoch Loss  18.748092651367188
Epoch Number :  752
	 Training accuracy:  99.61616161616162
	 Validation accuracy  88.66666666666667
	 Epoch Loss  14.232183456420898
Epoch Number :  753
	 Training accuracy:  99.11111111111111
	 Validation accuracy  84.22222222222223
	 Epoch Loss  17.46814727783203
Epoch Number :  754
	 Training accuracy:  99.1919191919192
	 Validation accuracy  84.88888888888889
	 Epoch Loss  17.169677734375
Epoch Number :  755
	 Training accuracy:  99.4949494949495
	 Validation accuracy  88.44444444444444
	 Epoch Loss  14.237252235412598
Epoch Number :  756
	 Training accuracy:  99.27272727272727
	 Validation accuracy  84.22222222222223
	 Epoch Loss  15.630884170532227
Epoch Number :  757
	 Training accuracy:  98.88888888888889
	 Validat

Epoch Number :  813
	 Training accuracy:  99.6969696969697
	 Validation accuracy  85.77777777777777
	 Epoch Loss  12.637377738952637
Epoch Number :  814
	 Training accuracy:  99.53535353535354
	 Validation accuracy  85.55555555555556
	 Epoch Loss  12.028165817260742
Epoch Number :  815
	 Training accuracy:  99.81818181818181
	 Validation accuracy  88.44444444444444
	 Epoch Loss  12.487632751464844
Epoch Number :  816
	 Training accuracy:  99.45454545454545
	 Validation accuracy  86.66666666666667
	 Epoch Loss  11.982217788696289
Epoch Number :  817
	 Training accuracy:  99.45454545454545
	 Validation accuracy  88.22222222222223
	 Epoch Loss  13.162150382995605
Epoch Number :  818
	 Training accuracy:  99.91919191919192
	 Validation accuracy  87.77777777777777
	 Epoch Loss  6.061595916748047
Epoch Number :  819
	 Training accuracy:  99.95959595959596
	 Validation accuracy  87.33333333333333
	 Epoch Loss  4.906916618347168
Epoch Number :  820
	 Training accuracy:  99.8989898989899
	 Vali

Epoch Number :  876
	 Training accuracy:  99.4949494949495
	 Validation accuracy  86.88888888888889
	 Epoch Loss  12.020727157592773
Epoch Number :  877
	 Training accuracy:  99.29292929292929
	 Validation accuracy  86.0
	 Epoch Loss  16.292827606201172
Epoch Number :  878
	 Training accuracy:  99.23232323232324
	 Validation accuracy  84.66666666666667
	 Epoch Loss  17.65713119506836
Epoch Number :  879
	 Training accuracy:  99.4949494949495
	 Validation accuracy  84.44444444444444
	 Epoch Loss  24.178627014160156
Epoch Number :  880
	 Training accuracy:  99.57575757575758
	 Validation accuracy  84.22222222222223
	 Epoch Loss  22.873300552368164
Epoch Number :  881
	 Training accuracy:  99.5959595959596
	 Validation accuracy  84.0
	 Epoch Loss  18.138023376464844
Epoch Number :  882
	 Training accuracy:  99.51515151515152
	 Validation accuracy  83.33333333333333
	 Epoch Loss  23.81692886352539
Epoch Number :  883
	 Training accuracy:  99.35353535353535
	 Validation accuracy  84.6666666

Epoch Number :  939
	 Training accuracy:  99.21212121212122
	 Validation accuracy  85.11111111111111
	 Epoch Loss  39.10798645019531
Epoch Number :  940
	 Training accuracy:  99.37373737373737
	 Validation accuracy  86.22222222222223
	 Epoch Loss  20.474830627441406
Epoch Number :  941
	 Training accuracy:  98.9090909090909
	 Validation accuracy  84.88888888888889
	 Epoch Loss  37.49427032470703
Epoch Number :  942
	 Training accuracy:  99.35353535353535
	 Validation accuracy  84.88888888888889
	 Epoch Loss  39.95133972167969
Epoch Number :  943
	 Training accuracy:  99.15151515151516
	 Validation accuracy  85.11111111111111
	 Epoch Loss  72.51661682128906
Epoch Number :  944
	 Training accuracy:  98.74747474747475
	 Validation accuracy  84.22222222222223
	 Epoch Loss  71.23975372314453
Epoch Number :  945
	 Training accuracy:  99.11111111111111
	 Validation accuracy  84.0
	 Epoch Loss  60.51457977294922
Epoch Number :  946
	 Training accuracy:  98.86868686868686
	 Validation accuracy 

# NETWORK SUMMARY & RESULTS

In [7]:
train_accuracy = 100*(Ntrain*Noutputs-np.array(train_errors))/(Ntrain*Noutputs)
val_accuracy = 100*(Nvalidation*Noutputs-np.array(validation_errors))/(Nvalidation*Noutputs)

stddev_train_errors = np.std(train_accuracy, axis=0)
stddev_val_errors = np.std(val_accuracy, axis=0)

mean_train_errors = np.mean(train_accuracy, axis=0)
mean_val_errors = np.mean(val_accuracy, axis=0)

epoch = list(range(1000))
plt.plot(epoch, mean_train_errors)
plt.plot(epoch, mean_val_errors)
plt.fill_between(epoch, mean_train_errors+stddev_train_errors, mean_train_errors-stddev_train_errors, alpha=0.5)
plt.fill_between(epoch, mean_val_errors+stddev_val_errors, mean_val_errors-stddev_val_errors, alpha=0.5)
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy in %')
plt.legend(['train', 'validation', 'test'])

print("Training accuracy {:4.3g}%+-{}".format(mean_train_errors[-1], stddev_train_errors[-1]))
print("Validation accuracy {:4.3g}%+-{}".format(mean_val_errors[-1], stddev_val_errors[-1]))

Training accuracy 99.7%+-0.0
Validation accuracy 87.1%+-0.0
